<a href="https://colab.research.google.com/github/prashshr/googlecolab/blob/main/Strategy_Dips1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

# ==========================================================
# 1. Robust loader for Close prices (split-adjusted, tz-naive)
# ==========================================================

def load_close(ticker, start="2015-01-01"):
    df = yf.download(
        ticker,
        start=start,
        auto_adjust=True,
        progress=False,
        group_by="column"
    )

    # Flatten possible MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(-1)

    if "Close" not in df.columns or df.empty:
        hist = yf.Ticker(ticker).history(start=start, auto_adjust=True)
        if isinstance(hist.columns, pd.MultiIndex):
            hist.columns = hist.columns.get_level_values(-1)
        if "Close" not in hist.columns or hist.empty:
            raise ValueError(f"No usable Close data for {ticker}")
        close = hist["Close"]
    else:
        close = df["Close"]

    close = close.astype(float).dropna()

    if getattr(close.index, "tz", None) is not None:
        close.index = close.index.tz_localize(None)

    return close


# ==========================================================
# 1b. OHLCV loader for technicals (auto_adjusted, tz-naive)
#      (ROBUST VERSION with history() fallback)
# ==========================================================

def load_ohlcv(ticker, start="2015-01-01"):
    needed = ["Open", "High", "Low", "Close", "Volume"]

    df = yf.download(
        ticker,
        start=start,
        auto_adjust=True,
        progress=False,
        group_by="column"
    )

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(-1)

    # If download is empty or missing columns, fall back to history()
    if df.empty or not set(needed).issubset(df.columns):
        hist = yf.Ticker(ticker).history(start=start, auto_adjust=True)
        if isinstance(hist.columns, pd.MultiIndex):
            hist.columns = hist.columns.get_level_values(-1)

        if hist.empty or not set(needed).issubset(hist.columns):
            missing = [c for c in needed if c not in hist.columns]
            raise ValueError(f"Missing columns {missing} for {ticker} from both download() and history()")

        df = hist[needed].astype(float)
    else:
        df = df[needed].astype(float)

    if getattr(df.index, "tz", None) is not None:
        df.index = df.index.tz_localize(None)

    return df


# ==========================================================
# 1c. RSI helper
# ==========================================================

def compute_rsi(close, period=14):
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


# ==========================================================
# 2. B1.2 signal detection (15%,20%,25% drawdowns from ATH)
# ==========================================================

def detect_bottoms_b1(close, thresholds):
    """
    thresholds = list of drawdown percentages from ATH
                 example: [0.15, 0.20, 0.25]
    Returns list of (date, price, ath_pre_correction)
    """
    ath = -np.inf
    hit = {}
    signals = []

    for date, price in close.items():
        price = float(price)

        if price > ath:
            ath = price
            hit = {thr: False for thr in thresholds}
            continue

        if ath <= 0:
            continue

        dd = (ath - price) / ath

        for thr in thresholds:
            if not hit[thr] and dd >= thr:
                signals.append((date, price, ath))
                hit[thr] = True

    return signals


# ==========================================================
# 3. Portfolio engine (with low-marker profit pool, no reinvest on normal buys)
# ==========================================================

class Cycle:
    def __init__(self, buy_price, shares, is_heavy=False):
        self.buy_price = buy_price
        self.shares = shares
        self.is_heavy = is_heavy   # heavy-buy cycles are never sold


class Portfolio:
    def __init__(self):
        self.cycles = []
        self.trade_log = []
        self.profit_booked = 0.0
        self.invested = 0.0

        # profits from normal sells accumulate here
        # and are only deployed at the next low-marker heavy buy.
        self.low_marker_pool = 0.0

    def buy(self, date, price, base_amount, max_cap):
        """
        Normal buy:
        - Uses ONLY base_amount (250, 500, 750).
        - Does NOT use the low_marker_pool.
        - Respects max_cap (normal cap).
        """
        amount = base_amount

        # respect hard cap for normal buys
        if self.invested + amount > max_cap:
            amount = max(0, max_cap - self.invested)

        if amount <= 0:
            return

        shares = amount / price
        self.cycles.append(Cycle(price, shares, is_heavy=False))
        self.invested += amount

        self.trade_log.append({
            "type": "BUY",
            "date": pd.Timestamp(date).to_pydatetime().replace(tzinfo=None),
            "price": float(price),
            "amount": float(amount),
            "shares": float(shares),
            "shares_sold": None,
            "realized": None,
            "profit": None,
        })

    def heavy_buy(self, date, price, base_amount):
        """
        Heavy capitulation buy:
        - Amount = base_amount (e.g. 1000 USD) + accumulated low_marker_pool.
        - Never sold (is_heavy=True).
        - DOES count toward 'invested' (for reporting).
        """
        amount = base_amount + self.low_marker_pool
        self.low_marker_pool = 0.0  # reset pool after deployment

        if amount <= 0:
            return

        shares = amount / price
        self.cycles.append(Cycle(price, shares, is_heavy=True))
        self.invested += amount

        self.trade_log.append({
            "type": "HEAVY_BUY",
            "date": pd.Timestamp(date).to_pydatetime().replace(tzinfo=None),
            "price": float(price),
            "amount": float(amount),
            "shares": float(shares),
            "shares_sold": None,
            "realized": None,
            "profit": None,
        })

    def sell_fraction(self, date, price, fraction):
        """
        Sells 'fraction' of TOTAL *normal* shares (is_heavy=False),
        pro-rata across normal cycles.

        Heavy-buy cycles (is_heavy=True) are NEVER sold.
        Realized profit from these sells goes into low_marker_pool.
        """
        total_shares = sum(c.shares for c in self.cycles if not c.is_heavy)
        if total_shares <= 0:
            return 0.0

        target = total_shares * fraction
        remaining = target
        realized_total = 0.0
        profit_total = 0.0

        for c in self.cycles:
            if c.is_heavy:
                continue  # do not touch heavy cycles

            if remaining <= 0 or c.shares <= 0:
                continue

            sell_here = min(c.shares, remaining)
            realized = sell_here * price
            cost = sell_here * c.buy_price
            profit = realized - cost

            c.shares -= sell_here
            remaining -= sell_here

            realized_total += realized
            profit_total += profit

        # profit from normal sells is booked and saved for next low-marker heavy buy
        self.low_marker_pool += profit_total
        self.profit_booked += profit_total

        self.trade_log.append({
            "type": "SELL",
            "date": pd.Timestamp(date).to_pydatetime().replace(tzinfo=None),
            "price": float(price),
            "amount": None,
            "shares": None,
            "shares_sold": float(target - remaining),
            "realized": float(realized_total),
            "profit": float(profit_total),
        })

        # remove empty cycles
        self.cycles = [c for c in self.cycles if c.shares > 1e-12]
        return realized_total

    def current_value(self, price):
        # heavy + normal cycles; all are part of portfolio value
        return sum(c.shares * price for c in self.cycles)


# ==========================================================
# 4. XIRR
# ==========================================================

def compute_xirr(cashflows):
    if not cashflows:
        return np.nan

    flows = []
    for d, amt in cashflows:
        d = pd.Timestamp(d).to_pydatetime().replace(tzinfo=None)
        flows.append((d, amt))

    flows = sorted(flows, key=lambda x: x[0])
    t0 = flows[0][0]

    def npv(rate):
        s = 0.0
        for d, amt in flows:
            years = (d - t0).days / 365.0
            s += amt / ((1 + rate) ** years)
        return s

    low, high = -0.999, 5.0
    npv_low = npv(low)
    npv_high = npv(high)

    if npv_low * npv_high > 0:
        return np.nan

    for _ in range(100):
        mid = (low + high) / 2.0
        val = npv(mid)
        if abs(val) < 1e-6:
            return mid
        if npv_low * val < 0:
            high = mid
            npv_high = val
        else:
            low = mid
            npv_low = val
    return mid


# ==========================================================
# 5. Strategy runner (with log-ATR marker + relaxed thresholds)
# ==========================================================

def run_strategy_b1_2(ticker, start="2021-01-01"):
    # Base price series
    close = load_close(ticker, start)

    # OHLCV for technicals (aligned to close index)
    ohlcv = load_ohlcv(ticker, start)
    ohlcv = ohlcv.reindex(close.index)

    high = ohlcv["High"]
    low = ohlcv["Low"]
    volume = ohlcv["Volume"]

    # 200-day EMA (currently SMA; can swap to EMA if you want)
    ema200 = close.rolling(200).mean()

    # RSI(14)
    rsi14 = compute_rsi(close, period=14)

    # -------- Upgrade D: log-based True Range & ATR-style measure --------
    log_close = np.log(close)
    log_high = np.log(high)
    log_low = np.log(low)
    log_prev_close = log_close.shift(1)

    lr1 = (log_high - log_low).abs()
    lr2 = (log_high - log_prev_close).abs()
    lr3 = (log_low - log_prev_close).abs()
    log_tr = pd.concat([lr1, lr2, lr3], axis=1).max(axis=1)

    vol_60 = volume.rolling(60).mean()
    log_tr_60 = log_tr.rolling(60).mean()

    # Drawdown from ATH
    ath_series = close.cummax()
    drawdown = (ath_series - close) / ath_series

    # Intraday close-location value (CLV)
    day_range = (high - low).replace(0, np.nan)
    clv = (close - low) / day_range  # can be NaN when no range

    # -------- RELAXED LOW-MARKER CONDITIONS (slightly easier to hit) --------
    # A) Drawdown_from_ATH ≥ 30%  (was 35%)
    # B) Price ≤ 200d_EMA * 0.85  (was 0.80)
    # C) Volume_today ≥ 1.3 × Volume_60d_avg (was 1.5)
    # D) logATR_today ≥ 1.3 × logATR_60d_avg (was 1.5)
    # E) (Close - Low) / (High - Low) ≥ 0.4 (was 0.5)
    # F) RSI(14) ≤ 40 (was 35)

    condA = drawdown >= 0.30
    condB = close <= ema200 * 0.85
    condC = volume >= 1.3 * vol_60
    condD = log_tr >= 1.3 * log_tr_60
    condE = clv >= 0.40
    condF = rsi14 <= 40.0

    low_marker = condA & condB & condC & condD & condE & condF
    marker_dates = low_marker[low_marker].index
    marker_count = int(low_marker.sum())

    # -------- Normal B1.2 ladder logic (unchanged thresholds / TP) --------
    thresholds = [0.15, 0.20, 0.25]        # drawdown levels from ATH
    tp_levels  = [1.20, 1.40, 1.60]        # take-profit levels (x ATH)
    tp_fracs   = [0.15, 0.15, 0.15]        # 15% of normal shares each TP

    signals = detect_bottoms_b1(close, thresholds)
    p = Portfolio()

    MAX_CAP = 3000.0  # cap only for normal buys

    for date, price, ath_pre in signals:
        # Trend filter
        if pd.isna(ema200.loc[date]) or price < ema200.loc[date]:
            continue

        # Buy size selection: 250 / 500 / 750 based on DD
        dd = (ath_pre - price) / ath_pre
        if dd >= 0.25:
            base_amt = 750.0
        elif dd >= 0.20:
            base_amt = 500.0
        else:
            base_amt = 250.0

        p.buy(date, price, base_amt, MAX_CAP)

        # Take-profits based on ATH * multiples
        future = close[close.index > date]

        for tp_mult, frac in zip(tp_levels, tp_fracs):
            target = ath_pre * tp_mult
            hit = future[future >= target]
            if hit.empty:
                break
            hit_date = hit.index[0]
            hit_price = hit.iloc[0]

            p.sell_fraction(hit_date, hit_price, frac)
            future = future[future.index > hit_date]

    # -------- Heavy low-marker buys --------
    HEAVY_BASE = 1000.0  # base amount for each low-marker
    for d in marker_dates:
        price = close.loc[d]
        # invest 1000 + accumulated low-marker pool
        p.heavy_buy(d, price, HEAVY_BASE)

    # -------- Final valuation + XIRR --------
    last_price = float(close.iloc[-1])
    last_date = close.index[-1]

    held_value = p.current_value(last_price)
    final_value = held_value + p.profit_booked
    total_pnl = final_value - p.invested

    # Per-ticker cashflows (for per-ticker XIRR)
    cashflows = []
    for log in p.trade_log:
        if log["type"] in ("BUY", "HEAVY_BUY"):
            cashflows.append((log["date"], -log["amount"]))
        elif log["type"] == "SELL":
            cashflows.append((log["date"], log["realized"]))
    if held_value > 0:
        cashflows.append(
            (pd.Timestamp(last_date).to_pydatetime().replace(tzinfo=None), held_value)
        )

    xirr_decimal = compute_xirr(cashflows)
    xirr_pct = xirr_decimal * 100.0 if pd.notna(xirr_decimal) else np.nan

    return {
        "ticker": ticker,
        "num_signals": len(signals),
        "invested": p.invested,
        "profit_booked": p.profit_booked,
        "held_value": held_value,
        "final_value": final_value,
        "total_pnl": total_pnl,
        "xirr": xirr_pct,      # store as percentage now
        "trade_log": p.trade_log,
        "marker_count": marker_count,
        "last_date": pd.Timestamp(last_date).to_pydatetime().replace(tzinfo=None),
    }


# ==========================================================
# 6. Run for all tickers + exports + portfolio stats
# ==========================================================

if __name__ == "__main__":
    tickers = ["NVDA","MSFT","PLTR","TSLA","AMZN","ASML","CRWD","META","AVGO","NOW"]

    print("Running B1.2 backtest...\n")
    results = []
    for t in tickers:
        print(f"Processing {t} ...")
        r = run_strategy_b1_2(t)
        results.append(r)

    summary = pd.DataFrame([{
        "ticker": r["ticker"],
        "num_signals": r["num_signals"],
        "invested": r["invested"],
        "profit_booked": r["profit_booked"],
        "held_value": r["held_value"],
        "final_value": r["final_value"],
        "total_pnl": r["total_pnl"],
        "xirr": r["xirr"],  # already in %
    } for r in results])

    print("\n======== FINAL SUMMARY (B1.2) ========\n")
    print(summary.to_string(index=False))

    # ----- LOW-MARKER COUNTS -----
    marker_counts = pd.DataFrame([{
        "ticker": r["ticker"],
        "low_marker_triggers": r["marker_count"],
    } for r in results])

    print("\n====== LOW MARKER COUNTS ======\n")
    print(marker_counts.to_string(index=False))

    # ----- Build trade log dataframe -----
    trade_rows = []
    for r in results:
        for log in r["trade_log"]:
            row = {"ticker": r["ticker"]}
            row.update(log)
            trade_rows.append(row)

    trade_df = pd.DataFrame(trade_rows)
    if "date" in trade_df.columns:
        trade_df["date"] = pd.to_datetime(trade_df["date"]).dt.tz_localize(None)

    # ----- Portfolio-wide stats (combined cashflow) -----
    total_invested = sum(r["invested"] for r in results)
    total_profit_booked = sum(r["profit_booked"] for r in results)
    total_held_value = sum(r["held_value"] for r in results)
    total_final_value = total_profit_booked + total_held_value
    total_pnl = total_final_value - total_invested

    # Combined cashflows (treat as one portfolio)
    combined_cf = []
    for r in results:
        for log in r["trade_log"]:
            if log["type"] in ("BUY", "HEAVY_BUY"):
                combined_cf.append((log["date"], -log["amount"]))
            elif log["type"] == "SELL":
                combined_cf.append((log["date"], log["realized"]))

    # Single terminal cashflow at the global end date
    if results:
        portfolio_end = max(r["last_date"] for r in results)
        combined_cf.append((portfolio_end, total_held_value))
        portfolio_xirr_dec = compute_xirr(combined_cf)
        portfolio_xirr_pct = portfolio_xirr_dec * 100.0 if pd.notna(portfolio_xirr_dec) else np.nan
    else:
        portfolio_xirr_pct = np.nan

    print("\n====== PORTFOLIO-WIDE STATS (COMBINED CASHFLOW) ======\n")
    print(f"Total invested     : {total_invested:,.2f}")
    print(f"Total profit_booked: {total_profit_booked:,.2f}")
    print(f"Total held_value   : {total_held_value:,.2f}")
    print(f"Total final_value  : {total_final_value:,.2f}")
    print(f"Total PnL          : {total_pnl:,.2f}")
    if pd.notna(portfolio_xirr_pct):
        print(f"Portfolio XIRR     : {portfolio_xirr_pct:,.2f}%")
    else:
        print("Portfolio XIRR     : NaN")

    # ----- CSV / Excel exports -----
    summary.to_csv("strategy_results_b1_2.csv", index=False)
    trade_df.to_csv("trade_logs_b1_2.csv", index=False)
    summary.to_excel("strategy_results_b1_2.xlsx", index=False)
    trade_df.to_excel("trade_logs_b1_2.xlsx", index=False)

    marker_counts.to_csv("low_marker_counts_b1_2.csv", index=False)

    print("\nCSV/Excel reports saved:")
    print(" - strategy_results_b1_2.csv / .xlsx")
    print(" - trade_logs_b1_2.csv / .xlsx")
    print(" - low_marker_counts_b1_2.csv")

    print("""
========= STRATEGY B1.2 DESCRIPTION (UPDATED) =========

ENTRY LOGIC (B1.2):
  1) Track all-time-high (ATH) for each ticker.
  2) When price drops 15% below ATH: first BUY.
  3) When price drops 20% below ATH: second BUY.
  4) When price drops 25% below ATH: third BUY.
  5) Buy sizes (per ATH regime) = 250, 500, 750 USD.
  6) Only buy if price > 200-day EMA (trend filter, no catching falling knives).
  7) Hard cap: max invested per ticker = 3000 USD for normal buys.
  8) Realized profit from normal SELLs is accumulated in a low-marker pool
     (not recycled into the next normal buy).

LOW-MARKER HEAVY BUY (with log-ATR and relaxed thresholds):
  - On any day where ALL of the following hold:
      * Drawdown_from_ATH ≥ 30%
      * Price ≤ 200d_EMA * 0.85
      * Volume_today ≥ 1.3 × Volume_60d_avg
      * logATR_today ≥ 1.3 × logATR_60d_avg
      * (Close - Low) / (High - Low) ≥ 0.4
      * RSI(14) ≤ 40
    then:
      * Execute a heavy BUY of (1000 USD + accumulated low-marker pool).
      * Heavy-buy cycles are never sold and do not affect the 3000 USD cap
        for normal buys, though the invested amount is still counted in
        'invested' for reporting.

EXIT / PROFIT-TAKING:
  - For each normal BUY, define take-profit levels at:
      * 1.20 × ATH before correction
      * 1.40 × ATH before correction
      * 1.60 × ATH before correction
  - At each target hit, sell these fractions of TOTAL *normal* shares:
      * 15% of total shares
      * 15% of total shares
      * 15% of total shares
  - Remaining normal shares plus all heavy-buy shares are held until the end.
  - Realized profit from these normal sells feeds into the low-marker pool.

NOTES:
  - 'xirr' in the summary is expressed in PERCENT (e.g. 76.71 = 76.71%).
  - Portfolio-wide XIRR is computed using a combined cashflow method:
      * all BUY / HEAVY_BUY as negative cashflows,
      * all SELL realized amounts as positive cashflows,
      * one final positive cashflow equal to the total held_value at the end date.
""")

Running B1.2 backtest...

Processing NVDA ...
Processing MSFT ...
Processing PLTR ...
Processing TSLA ...
Processing AMZN ...
Processing ASML ...
Processing CRWD ...
Processing META ...
Processing AVGO ...
Processing NOW ...

======== FINAL SUMMARY (B1.2) ========

ticker  num_signals     invested  profit_booked    held_value   final_value     total_pnl       xirr
  NVDA           14 11091.661364    4091.661364  86242.907539  90334.568903  79242.907539  98.897180
  MSFT            5     0.000000       0.000000      0.000000      0.000000      0.000000        NaN
  PLTR           12 11622.941308    1622.941308 129628.623739 131251.565047 119628.623739 108.572920
  TSLA            9 14000.000000       0.000000  28788.435249  28788.435249  14788.435249  31.672470
  AMZN            7  5250.000000       0.000000   9953.377023   9953.377023   4703.377023  21.607261
  ASML            8  9000.000000       0.000000  16323.812910  16323.812910   7323.812910  27.420808
  CRWD           15 14048.5